# S/N budget estimator and exposure time calculator, adapted from Turyshev2025

In [1]:
import numpy as np

In [2]:
# Fundamental physical constants

h = 6.62607015e-34      # [J s ≡ J Hz^-1] Planck's constant
c = 299792458.          # [m/s] Light speed in vacuum

# Useful conversion factors

arcsec_per_radian = 3600*180/np.pi

#### Signal estimation

In [3]:
def F_0_V(F_lambda_0_W_m_2_nm_1=3.631e-11, lam_nm=550.):    # [W m^-2 nm^-1] ≡ [erg s^-1 cm^-2 Å^-1]

    """Adapted from Eq. (7) of Turyshev2025.
       Spectral flux density zero point for calibration from: https://www.astronomy.ohio-state.edu/martini.10/usefuldata.html."""

    lam_m = lam_nm * 1.e-9
    F_0_V_photons_m_2_s_1_nm_1 = F_lambda_0_W_m_2_nm_1 * lam_m / (h * c)

    return F_0_V_photons_m_2_s_1_nm_1

F_0_V()

100533824.9121117

In [4]:
def apparent_magnitude_to_flux(F_0point, mag):

    return F_0point * 10**(-0.4 * mag)

In [5]:
def F_exoworld(F_0_V_photons_m_2_s_1_nm_1, mV_exoworld=27.77):

    """Adapted from Eq. (8) of Turyshev2025."""

    F_exoworld_photons_m_2_s_1_nm_1 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_exoworld)

    return F_exoworld_photons_m_2_s_1_nm_1

F_exoworld(F_0_V())

0.0007839930379253792

In [6]:
def spectral_flux_density_exoworld(F_0_V_photons_m_2_s_1_nm_1, mV_exoworld=27.77, A_m2=100., dlam_nm=50.):

    """Adapted from Eq. (9) of Turyshev2025."""

    spectral_flux_density_exoworld_photons_m_2_s_1_nm_1 = F_exoworld(F_0_V_photons_m_2_s_1_nm_1, mV_exoworld) * A_m2 * dlam_nm

    return spectral_flux_density_exoworld_photons_m_2_s_1_nm_1

spectral_flux_density_exoworld(F_0_V())

3.919965189626896

In [7]:
spectral_flux_density_exoworld_photons_m_2_s_1_nm_1_pixel_1 = spectral_flux_density_exoworld(F_0_V()) / 100

spectral_flux_density_exoworld_photons_m_2_s_1_nm_1_pixel_1

0.03919965189626896

#### Astrophysical background sources

In [8]:
def F_zodi(F_0_V_photons_m_2_s_1_nm_1, mV_zodi_arcsec_2=23.):

    """Adapted from Turyshev2025."""

    F_zodi_photons_m_2_s_1_nm_1_arcsec_2 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_zodi_arcsec_2)

    return F_zodi_photons_m_2_s_1_nm_1_arcsec_2

F_zodi(F_0_V())

0.06343255519698253

In [9]:
def F_exozodi(F_0_V_photons_m_2_s_1_nm_1, mV_exozodi_arcsec_2=23.):

    """Adapted from Turyshev2025.
       Assuming 8 times the Solar System's zodiacal cloud, from the 5--8 upper limit factor for a disk coplanar with the α Cen AB orbit, as reported by Beichman+2025 and Sanghi+2025."""

    F_exozodi_photons_m_2_s_1_nm_1_arcsec_2 = 8. * apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_exozodi_arcsec_2)

    return F_exozodi_photons_m_2_s_1_nm_1_arcsec_2

F_exozodi(F_0_V())

0.5074604415758602

In [10]:
def F_diffuse_Milky_Way(F_0_V_photons_m_2_s_1_nm_1, mV_MilkyWay_arcsec_2=24.):

    """Adapted from Turyshev2025."""

    F_MilkyWay_photons_m_2_s_1_nm_1_arcsec_2 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_MilkyWay_arcsec_2)

    return F_MilkyWay_photons_m_2_s_1_nm_1_arcsec_2

F_diffuse_Milky_Way(F_0_V())

0.025252955070449234

In [11]:
def F_extragalactic_background(F_0_V_photons_m_2_s_1_nm_1, mV_extragalactic_arcsec_2=26.5):

    """Adapted from Turyshev2025."""

    F_extragalactic_photons_m_2_s_1_nm_1_arcsec_2 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_extragalactic_arcsec_2)

    return F_extragalactic_photons_m_2_s_1_nm_1_arcsec_2

F_extragalactic_background(F_0_V())

0.0025252955070449237

In [12]:
def F_unresolved_MW_stars(F_0_V_photons_m_2_s_1_nm_1, mV_unresolved_stars=27.5):

    """Adapted from Turyshev2025."""

    F_unresolved_stars_photons_m_2_s_1_nm_1_arcsec_2 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_unresolved_stars)

    return F_unresolved_stars_photons_m_2_s_1_nm_1_arcsec_2

F_unresolved_MW_stars(F_0_V())

0.001005338249121117

In [13]:
def F_unresolved_extragalactic_point_sources(F_0_V_photons_m_2_s_1_nm_1, mV_unresolved_extragalactic_point_sources=28.):

    """Adapted from Turyshev2025."""

    F_unresolved_extragalactic_point_sources_photons_m_2_s_1_nm_1_arcsec_2 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_unresolved_extragalactic_point_sources)

    return F_unresolved_extragalactic_point_sources_photons_m_2_s_1_nm_1_arcsec_2

F_unresolved_extragalactic_point_sources(F_0_V())

0.0006343255519698254

In [14]:
F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2 = np.sum([F_zodi(F_0_V()), F_exozodi(F_0_V()),
                                     F_diffuse_Milky_Way(F_0_V()), F_unresolved_MW_stars(F_0_V()),
                                     F_extragalactic_background(F_0_V()), F_unresolved_extragalactic_point_sources(F_0_V())])

F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2

np.float64(0.6003109111514279)

In [15]:
def Airy_core_solid_angle_arcsec2(D_m=2*np.sqrt(100./np.pi), lam_nm=550.):

    """Adapted from Turyshev2025."""

    lam_m = lam_nm * 1e-9
    theta_rad = 1.22 * lam_m / D_m
    theta_arcsec = theta_rad * arcsec_per_radian
    Omega_core_arcsec2 = np.pi * (theta_arcsec**2)

    return Omega_core_arcsec2

Airy_core_solid_angle_arcsec2()

np.float64(0.0004726449921600001)

In [16]:
def background_photon_count_rate_photons_s_1(F_background_photons_m_2_s_1_nm_1_arcsec_2, A_m2=100., D_m=2*np.sqrt(100/np.pi), dlam_nm=50., lam_nm=550.):

    """Adapted from Eq. (12) of Turyshev2025."""

    background_photon_count_rate_photons_s_1 = F_background_photons_m_2_s_1_nm_1_arcsec_2 * A_m2 * dlam_nm * Airy_core_solid_angle_arcsec2(D_m, lam_nm)

    return background_photon_count_rate_photons_s_1

background_photon_count_rate_photons_s_1(F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2)

np.float64(1.4186697294736457)

#### Detector noise

In [17]:
dark_current_electrons_s_1_pixel = 1.e-4

dark_current_electrons_s_1_pixel

0.0001

In [18]:
CIC_rate_electrons_frame_1_pixel_1 = 5 * 1.e-3
N_pixels_photometric_aperture = 4.
frame_rate_s_1 = 10.
CIC_rate_electrons_frame_1_s_1 = N_pixels_photometric_aperture * CIC_rate_electrons_frame_1_pixel_1 * frame_rate_s_1

CIC_rate_electrons_frame_1_s_1

0.2

In [19]:
sigma_readout_electrons_frame_1_pixel_1 = .1    # [e-/frame/pixel]
N_frames = 100
# Wrong formula
readout_noise = N_pixels_photometric_aperture * (sigma_readout_electrons_frame_1_pixel_1**2) / N_frames

readout_noise

0.0004000000000000001

In [20]:
detector_noise_electrons_s_1 = dark_current_electrons_s_1_pixel + CIC_rate_electrons_frame_1_s_1 + readout_noise

detector_noise_electrons_s_1

0.2005

#### Stellar leakage

In [21]:
def stellar_leakage_count_rate_s_1(F_0_V_photons_m_2_s_1_nm_1, mV_star=4.83, A_m2=100., dlam_nm=50., residual_raw_C=1e-8):

    """Adapted from Eq. (13) of Turyshev2025."""

    F_star_photons_m_2_s_1_nm_1 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_star)

    stellar_leakage_counts_photons_s_1 = F_star_photons_m_2_s_1_nm_1 * A_m2 * dlam_nm * residual_raw_C

    return stellar_leakage_counts_photons_s_1

stellar_leakage_count_rate_s_1(F_0_V())

58.78712350580647

In [22]:
def quasistatic_speckle_drift(F_0_V_photons_m_2_s_1_nm_1, mV_star=4.83, A_m2=100., dlam_nm=50.):

    """Adapted from Eq. (14) of Turyshev2025."""

    F_star_photons_m_2_s_1_nm_1 = apparent_magnitude_to_flux(F_0_V_photons_m_2_s_1_nm_1, mV_star)

    raw_C_degradation = 1e-9

    quasistatic_speckle_drift_counts_photons_s_1 = np.sqrt(F_star_photons_m_2_s_1_nm_1 * A_m2 * dlam_nm * raw_C_degradation)

    return quasistatic_speckle_drift_counts_photons_s_1

quasistatic_speckle_drift(F_0_V())

np.float64(2.4246056072237083)

#### Other miscellanea: pointing jitter, thermal emission, quantization noise, and cosmic rays

In [23]:
pointing_jitter_count_rate_photons_s_1 = .3 * (stellar_leakage_count_rate_s_1(F_0_V()) + background_photon_count_rate_photons_s_1(F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2))

pointing_jitter_count_rate_photons_s_1

np.float64(18.061737970584034)

In [24]:
# def thermal_emission_primary_mirror    # Above ~800 nm

In [25]:
# def quantization_noise

In [26]:
# def cosmic_ray_count_rate_photons_s_1

## S/N and exposure time calculator

In [27]:
throughput_optics_instrumentation = .2
duty_cycle = .5
eta_system = throughput_optics_instrumentation * duty_cycle

eta_system

0.1

In [28]:
CR_signal = eta_system * (spectral_flux_density_exoworld(F_0_V()) +
                          background_photon_count_rate_photons_s_1(F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2) +
                          stellar_leakage_count_rate_s_1(F_0_V()))

CR_signal

np.float64(6.412575842490701)

In [29]:
CR_instrumentation = quasistatic_speckle_drift(F_0_V()) + pointing_jitter_count_rate_photons_s_1

CR_noise = detector_noise_electrons_s_1 + CR_instrumentation + eta_system * (background_photon_count_rate_photons_s_1(F_astrophysical_background_photons_m_2_s_1_nm_1_arcsec_2) +
                                                                             stellar_leakage_count_rate_s_1(F_0_V()))

CR_noise

np.float64(26.707422901335754)

In [30]:
CR_signal_per_pixel = CR_signal / 100

SNR_per_pixel = CR_signal_per_pixel / np.sqrt(CR_signal_per_pixel + CR_noise)

SNR_per_pixel

np.float64(0.012393551258014952)

In [31]:
SNR_detection_threshold = 5.

t_integration_s = (SNR_detection_threshold / SNR_per_pixel)**2

t_integration_hr = t_integration_s / 3600.

t_integration_hr

np.float64(45.21119365428371)